In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import cuml
import json
import joblib

from sklearn.model_selection import KFold, StratifiedKFold
from cuml.metrics import mean_squared_log_error
from xgboost import XGBRegressor

from catboost import CatBoostRegressor
from sklearn.preprocessing import KBinsDiscretizer

# Setting Matplotlib defaults
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(8,5), dpi=150)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')
plt.tight_layout()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 500)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/calories-burnt-prediction/calories.csv
/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


<Figure size 1200x750 with 0 Axes>

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv', index_col='id')
org = pd.read_csv('/kaggle/input/calories-burnt-prediction/calories.csv', index_col='User_ID')
org = org.rename(columns={'Gender': 'Sex'})

In [3]:
train = pd.concat([train, org], ignore_index=True)

# Data Understanding

In [4]:
train.shape

(765000, 8)

In [5]:
train.head(10)

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
5,female,26,156.0,56.0,19.0,100.0,40.5,103.0
6,female,21,172.0,73.0,3.0,81.0,38.3,9.0
7,male,46,188.0,94.0,23.0,100.0,40.8,145.0
8,female,33,166.0,63.0,25.0,107.0,40.5,161.0
9,male,65,185.0,88.0,23.0,104.0,41.0,185.0


In [6]:
train.describe()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000,765000.000000
mean,41.447255,174.693126,75.142162,15.423163,95.484672,40.036041,88.307424
std,15.213677,12.854173,14.004122,8.353421,9.452476,0.779863,62.396760
min,20.000000,123.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [7]:
train.isnull().sum()

Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [8]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 765000 entries, 0 to 764999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Sex         765000 non-null  object
 1   Age         765000 non-null  int64
 2   Height      765000 non-null  float64
 3   Weight      765000 non-null  float64
 4   Duration    765000 non-null  float64
 5   Heart_Rate  765000 non-null  float64
 6   Body_Temp   765000 non-null  float64
 7   Calories    765000 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 47.4+ MB


# Data Preprocessing

## Reducing memory usage

In [9]:
train['Age'] = train['Age'].astype('int8')
test['Age'] = test['Age'].astype('int8')

In [10]:
num_cols = test.select_dtypes(include='float64').columns
for col in num_cols:
    train[num_cols] = train[num_cols].astype('float32')
    test[num_cols] = test[num_cols].astype('float32')

In [11]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 765000 entries, 0 to 764999
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Sex         765000 non-null  object
 1   Age         765000 non-null  int8
 2   Height      765000 non-null  float32
 3   Weight      765000 non-null  float32
 4   Duration    765000 non-null  float32
 5   Heart_Rate  765000 non-null  float32
 6   Body_Temp   765000 non-null  float32
 7   Calories    765000 non-null  float64
dtypes: float32(5), float64(1), int8(1), object(1)
memory usage: 27.7+ MB


In [12]:
mapping = {'male': 1, 'female': 0}

train['Sex'] = train['Sex'].map(mapping).astype('int8')
test['Sex'] = test['Sex'].map(mapping).astype('int8')

In [13]:
def add_statistical_features(df, features):
    df_new = df.copy()
    df_new["row_mean"] = df[features].mean(axis=1)
    df_new["row_max"] = df[features].max(axis=1)
    df_new["row_min"] = df[features].min(axis=1)
    df_new["row_median"] = df[features].median(axis=1)
    df_new["row_skew"] = df[features].skew(axis=1)
    df_new["row_unique"] = df[features].nunique(axis=1)
    
    return df_new

In [14]:
features = ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

In [15]:
def add_features(df):
    for i, c1 in enumerate(features):
        for c2 in (features[i+1:]):
            m = f'{c1}*{c2}'
            df[m] = df[c1] * df[c2]
            a = f'{c1}+{c2}'
            df[a] = df[c1] + df[c2]
            s = f'{c1}-{c2}'
            df[a] = df[c1] - df[c2]
            d = f'{c1}/{c2}'
            df[d] = df[c1] / (df[c2] + 1e-3)
            rd = f'{c2}/{c1}'
            df[rd] = df[c2] / (df[c1] + 1e-3)
    return df

In [16]:
def feature_engineer(df):
    
    df['BMI'] = df['Weight'] / (df['Height']/100) ** 2
    
    Max_Heart_Rate = 207 - (0.7 * df['Age'])
    df['HR_Ratio'] = df['Heart_Rate'] / Max_Heart_Rate
    df['HR_Reserve'] =  df['HR_Ratio'] * df['Duration']
    df['Thermal_Load'] = df['Body_Temp'] * df['Duration'] * df['HR_Ratio']

    df['Anaerobic_Contribution'] = df['HR_Ratio']**2 * np.minimum(df['Duration'], 180) * df['Weight'] * 0.05
    
    MET_Estimate = (df['Heart_Rate'] / Max_Heart_Rate) * 15
    df['MET_Calories_Rate'] = MET_Estimate * 3.5 * df['Weight'] / 200
    
    df['EPOC_Factor'] = df['HR_Ratio']**2 * np.log1p(df['Duration']/10)
    df['MET_Calories_Total'] = df['MET_Calories_Rate'] * df['Duration']
    
    df['Exercise_Intensity_Index'] = df['Heart_Rate'] * df['Body_Temp'] / (df['Weight'] * df['Age'])
    df['Exercise_Economy'] = df['Heart_Rate'] / (df['Weight']**0.75)

    df['exertion_score'] = df['Duration'] * df['Heart_Rate'] * df['Body_Temp']
    df['Age_Adjusted_Exertion'] = df['exertion_score'] / (1 + 0.01 * (df['Age'] - 30))
    
    df['body_theta'] = np.arctan2(df['Weight'], df['Height'])
    
    return df

In [17]:
X = train.copy()
y = X.pop('Calories')
y = np.log1p(y)

X = add_features(X)
X = feature_engineer(X)
X = add_statistical_features(X, features=['Duration', 'Heart_Rate', 'Body_Temp', 'Weight'])

X_test = test.copy()
X_test = add_features(X_test)
X_test = feature_engineer(X_test)
X_test = add_statistical_features(X_test, features=['Duration', 'Heart_Rate', 'Body_Temp', 'Weight'])

In [18]:
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=100)

oof_xgb = np.zeros(len(y))
test_xgb = np.zeros(len(X_test))

params = {
    'n_estimators': 2000, 
    'learning_rate': 0.012429613949090264, 
    'max_depth': 9, 
    'min_child_weight': 11, 
    'alpha': 0.031799792818970524, 
    'reg_lambda': 0.3580931274078332, 
    'subsample': 0.8416119186946701, 
    'colsample_bytree': 0.5066402912386329, 
    'gamma': 1.037011350094906e-08,
    'grow_policy': 'lossguide'
}

for fold, (train_index, valid_index) in enumerate(kf.split(X, y), start=1):
        X_train, X_valid = X.iloc[train_index].copy(), X.iloc[valid_index].copy()
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        xgb = XGBRegressor(**params, early_stopping_rounds=100, tree_method='gpu_hist', device='cuda', seed=100, 
            objective='reg:squarederror', eval_metric='rmsle',
        ).fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    
        valid_pred = xgb.predict(X_valid)
        oof_xgb[valid_index] = valid_pred
        test_xgb += xgb.predict(X_test) / n_folds
    
        fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(valid_pred)))
        print(f"Fold {fold} RMSE: {fold_rmsle:.6f}")

test_xgb = np.expm1(test_xgb)
test_xgb = np.clip(test_xgb, 1, 314)

overall_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y), np.expm1(oof_xgb)))
print(f"\nOverall OOF RMSLE: {overall_rmsle:.6f}")   

Fold 1 RMSE: 0.058502
Fold 2 RMSE: 0.058860
Fold 3 RMSE: 0.059491
Fold 4 RMSE: 0.059488
Fold 5 RMSE: 0.058808

Overall OOF RMSLE: 0.059031


In [19]:
out_path_oof = "/kaggle/working/oof_xgb.pkl"
joblib.dump(oof_xgb, out_path_oof)

out_path_test = "/kaggle/working/test_xgb.pkl"
joblib.dump(test_xgb, out_path_test)

print(f"OOF predictions saved to: {out_path_oof}")

OOF predictions saved to: /kaggle/working/oof_xgb.pkl


In [20]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
sub['Calories'] = test_xgb
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(sub.head(5))

Your submission was successfully saved!
       id    Calories
0  750000   27.421792
1  750001  107.778877
2  750002   87.630953
3  750003  126.301407
4  750004   75.537072
